In [0]:
from pyspark.sql import SparkSession, types

spark = SparkSession.builder.appName('MyPractise').getOrCreate()
print(spark)

sc = spark.sparkContext
print(sc)

<SparkContext master=local[8] appName=Databricks Shell>


In [0]:
df = spark.read.format("text").load("/FileStore/tables/ghtorrent_logs.txt")

df.display()

print(df.count())

type(df)

value
"INFO, 2017-03-22T20:11:49+00:00, ghtorrent-31 -- ghtorrent.rb: Added pullreq_commit 244eeac28bf419642d5d5c3b388bd2999c8c72e6 to tgstation/tgstation -> 25341"
"DEBUG, 2017-03-23T11:15:14+00:00, ghtorrent-30 -- retriever.rb: Commit mzvast/FlappyFrog -> 80bf5c5fde7be6274a2721422f4d9a773583f73c exists"
"DEBUG, 2017-03-22T20:15:48+00:00, ghtorrent-35 -- ghtorrent.rb: Parent af8451e16e077f7c6cae3f98bf43bffaca562f88 for commit 2ef393531a3cfbecc69f17d2cedcc95662fae1e6 exists"
"DEBUG, 2017-03-24T12:29:50+00:00, ghtorrent-49 -- ghtorrent.rb: Parent cf060bf3b789ac6391b2f7c1cdc34191c2bc773d for commit 8c924c1115e1abddcaddc27c6e7fd5806583ea90 exists"
"DEBUG, 2017-03-23T09:00:44+00:00, ghtorrent-8 -- retriever.rb: Commit iamtheanon/d3 -> a7caf9375fe14d7235562af541fe9decf499bbfb exists"
"DEBUG, 2017-03-24T10:52:47+00:00, ghtorrent-50 -- ghtorrent.rb: Repo alyuev/urantia-study-edition exists"
"DEBUG, 2017-03-23T11:02:10+00:00, ghtorrent-45 -- retriever.rb: Commit heycalmdown/node-confluence -> 2900cc8718deb2453ff39c909d2563d96227fb3f exists"
"INFO, 2017-03-23T10:31:57+00:00, ghtorrent-45 -- retriever.rb: Added commit patrickTingen/DataDigger -> d807d38fe9b9bd1325d45d69752fbabd89d73d96"
"INFO, 2017-03-22T20:14:31+00:00, ghtorrent-32 -- ghtorrent.rb: Added commit_assoc of cfd33b7282e5b03e4dc5c2509564e2189cd69af7 with RicardoGuzmanVelasco/Unity-Beat-Detection"
"DEBUG, 2017-03-23T11:03:03+00:00, ghtorrent-39 -- ghtorrent.rb: Transaction committed (618 ms)"


281234
Out[36]: pyspark.sql.dataframe.DataFrame

In [0]:
df1= df.selectExpr("split(value, ',') as Text_Data_In_Rows_Using_format_load")

df1.display()
print(df1.count())

Text_Data_In_Rows_Using_format_load
"List(INFO, 2017-03-22T20:11:49+00:00, ghtorrent-31 -- ghtorrent.rb: Added pullreq_commit 244eeac28bf419642d5d5c3b388bd2999c8c72e6 to tgstation/tgstation -> 25341)"
"List(DEBUG, 2017-03-23T11:15:14+00:00, ghtorrent-30 -- retriever.rb: Commit mzvast/FlappyFrog -> 80bf5c5fde7be6274a2721422f4d9a773583f73c exists)"
"List(DEBUG, 2017-03-22T20:15:48+00:00, ghtorrent-35 -- ghtorrent.rb: Parent af8451e16e077f7c6cae3f98bf43bffaca562f88 for commit 2ef393531a3cfbecc69f17d2cedcc95662fae1e6 exists)"
"List(DEBUG, 2017-03-24T12:29:50+00:00, ghtorrent-49 -- ghtorrent.rb: Parent cf060bf3b789ac6391b2f7c1cdc34191c2bc773d for commit 8c924c1115e1abddcaddc27c6e7fd5806583ea90 exists)"
"List(DEBUG, 2017-03-23T09:00:44+00:00, ghtorrent-8 -- retriever.rb: Commit iamtheanon/d3 -> a7caf9375fe14d7235562af541fe9decf499bbfb exists)"
"List(DEBUG, 2017-03-24T10:52:47+00:00, ghtorrent-50 -- ghtorrent.rb: Repo alyuev/urantia-study-edition exists)"
"List(DEBUG, 2017-03-23T11:02:10+00:00, ghtorrent-45 -- retriever.rb: Commit heycalmdown/node-confluence -> 2900cc8718deb2453ff39c909d2563d96227fb3f exists)"
"List(INFO, 2017-03-23T10:31:57+00:00, ghtorrent-45 -- retriever.rb: Added commit patrickTingen/DataDigger -> d807d38fe9b9bd1325d45d69752fbabd89d73d96)"
"List(INFO, 2017-03-22T20:14:31+00:00, ghtorrent-32 -- ghtorrent.rb: Added commit_assoc of cfd33b7282e5b03e4dc5c2509564e2189cd69af7 with RicardoGuzmanVelasco/Unity-Beat-Detection)"
"List(DEBUG, 2017-03-23T11:03:03+00:00, ghtorrent-39 -- ghtorrent.rb: Transaction committed (618 ms))"


281234


In [0]:
RDD = sc.parallelize(df1)

display(RDD)

type(RDD)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-2245227721271034> in <module>
----> 1 RDD = sc.parallelize(df1)
      2 
      3 display(RDD)
      4 
      5 type(RDD)

/databricks/spark/python/pyspark/context.py in parallelize(self, c, numSlices)
    599                 return self._jvm.PythonParallelizeServer(self._jsc.sc(), numSlices)
    600 
--> 601             jrdd = self._serialize_to_jvm(c, serializer, reader_func, createRDDServer)
    602 
    603         return RDD(jrdd, self, serializer)

/databricks/spark/python/pyspark/context.py in _serialize_to_jvm(self, data, serializer, reader_func, createRDDServer)
    637             try:
    638                 try:
--> 639                     serializer.dump_stream(data, tempFile)
    640                 finally:
    641                     tempFile.close()

/databricks/spark/python/pyspark/serializers.py in dump_stream